In [11]:
import pygplates
import os
import numpy as np
#import moviepy.editor as mpy
#from call_system_command import call_system_command
from resolve_topologies import resolve_topologies as topology2gmt



In [10]:

#'''
#####################################
#MODELDIR = '/Users/simon/GIT/agegrid-dev//input_files/'
input_rotation_filenames = ['../MullerData/Global_EarthByte_230-0Ma_GK07_AREPS.rot']
topology_features = ['../MullerData/Global_EarthByte_230-0Ma_GK07_AREPS_PlateBoundaries.gpmlz',\
                          '../MullerData/Global_EarthByte_230-0Ma_GK07_AREPS_Topology_BuildingBlocks.gpmlz']
#####################################
input_continents_filename = '../MullerData/continental_polygons.gpmlz'
#input_coastlines_filename = '%s/StaticPolygons/Matthews_etal_GPC_2016_Coastlines.gpmlz' % MODELDIR



# --- set parameters
#min_time = 0.
#max_time = 100.

min_time = 100.
max_time = 220.

time_step = 10.  # time step for creating seed points

continent_color_list = np.linspace(0.0,1,time_list.shape[0])[::-1]

print continent_color_list

os.system('gmt gmtset COLOR_MODEL RGB MAP_FRAME_TYPE fancy FONT_ANNOT_PRIMARY 10 FONT_LABEL 8 FONT_TITLE 8 FONT_ANNOT_PRIMARY Helvetica FORMAT_GEO_MAP ddd')

#os.system('gmt makecpt -T%0.2f/%0.2f -Cdarkred,red,orange,gold -Z > subduction_age.cpt' % (min_time,max_time))
os.system('gmt makecpt -T%0.2f/%0.2f -Cplasma -I -Z > subduction_age.cpt' % (0,230))

# Start Plotting

# Select a projection - central meridian is set to 30 deg longitude for each
#proj='-Ji30/0.02i'  # uncomment for Sinusoidal projection
#proj='-JN30/5i'      # uncomment for Robinson projection
proj='-JW180/6c'    # uncomment for Mollweide projection


def plot_V2012_map(min_time,max_time,Xoffset,Yoffset):
    
    VDM2009_features = pygplates.FeatureCollection('../vanderMeerData/VDM2009_features.gpml')
    VDM2012_features = pygplates.FeatureCollection('../vanderMeerData/VDM2012_features.gpml')

    input_rotation_filenames = ['../MullerData/Global_EarthByte_230-0Ma_GK07_AREPS_VDM.rot']
    # Static polygons to determine whether subduction segment is adjacent to continent or not
    #input_continents_filename = '/Users/Simon/Work/DynamicTopography/MODELS/StaticPolygons/StaticPolygonsForPlateFrame2013.2.shp'
    #input_continents_filename = 'continental_polygons.gpmlz'


    VDM2009_recon_features = []
    VDM2012_recon_features = []

    for feature in VDM2009_features:
        if (np.mean(feature.get_valid_time())<max_time) & (np.mean(feature.get_valid_time())>min_time): 
            VDM2009_recon_features.append(feature)       
    pygplates.FeatureCollection(VDM2009_recon_features).write('./tmp/tmp1.gmt')        

    for feature in VDM2012_features:
        if (np.mean(feature.get_valid_time())<max_time) & (np.mean(feature.get_valid_time())>min_time): 
            VDM2012_recon_features.append(feature)       
    pygplates.FeatureCollection(VDM2012_recon_features).write('./tmp/tmp2.gmt')  

    time_list = np.arange(max_time,min_time-time_step,-time_step)
    continent_color_list = np.linspace(0.1,1,time_list.shape[0])[::-1]

    os.system('gmt makecpt -T%0.2f/%0.2f > continent_age.cpt' % (min_time,max_time))
    os.system('gmt psbasemap -R0/360/-90/90 %s -B+gwhite -X%0.5f -Y%0.5f -K -O >> %s ' % (proj,Xoffset,Yoffset,outfile))

    for reconstruction_time,color_ind in zip(time_list,continent_color_list):

        output_reconstructed_continents_filename = './tmp/continents.gmt'
        pygplates.reconstruct(input_continents_filename, input_rotation_filenames, output_reconstructed_continents_filename, reconstruction_time)

        #plot continents
        os.system('gmt psxy -R %s -G%d/%d/%d@60 ./tmp/continents.gmt -O -K -N >> %s' % (proj,225*color_ind,225*color_ind,225*color_ind,outfile))

    os.system('gmt psxy -R %s -W0.5p,2_0.5:0+cl -aZ=FROMAGE -O -K -m -Csubduction_age.cpt ./tmp/tmp2.gmt -V >> %s' % (proj,outfile))
    os.system('gmt psxy -R %s -W0.8p+cl -aZ=FROMAGE -O -K -m -Csubduction_age.cpt ./tmp/tmp1.gmt -V >> %s' % (proj,outfile))

    os.system('gmt psbasemap -R0/360/-90/90 %s -B45wE  --MAP_TICK_LENGTH=0.1c -O -K >> %s ' % (proj,outfile))    
    os.system('gmt psclip -C -O -K >> %s ' % (outfile))

    
def plot_M2016_map(min_time,max_time,Xoffset,Yoffset):

    #####################################
    #MODELDIR = '/Users/simon/GIT/agegrid-dev//input_files/'
    input_rotation_filenames = ['../MullerData/Global_EarthByte_230-0Ma_GK07_AREPS.rot']
    topology_features = ['../MullerData/Global_EarthByte_230-0Ma_GK07_AREPS_PlateBoundaries.gpmlz',\
                              '../MullerData/Global_EarthByte_230-0Ma_GK07_AREPS_Topology_BuildingBlocks.gpmlz']
    #####################################
    #input_continents_filename = 'continental_polygons.gpmlz'

    time_list = np.arange(max_time,min_time-time_step,-time_step)
    continent_color_list = np.linspace(0.1,1,time_list.shape[0])[::-1]
    
    os.system('gmt makecpt -T%0.2f/%0.2f > continent_age.cpt' % (min_time,max_time))
    os.system('gmt psbasemap -R0/360/-90/90 %s -B+gwhite -X%0.5f -Y%0.5f -O -K >> %s ' % (proj,Xoffset,Yoffset,outfile))

    for reconstruction_time,color_ind in zip(time_list,continent_color_list):

        output_reconstructed_continents_filename = './tmp/continents.gmt'
        pygplates.reconstruct(input_continents_filename, input_rotation_filenames, output_reconstructed_continents_filename, reconstruction_time)

        #plot continents
        os.system('gmt psxy -R %s -G%d/%d/%d@60 ./tmp/continents.gmt -O -K -N >> %s' % (proj,225*color_ind,225*color_ind,225*color_ind,outfile))

    for reconstruction_time,color_ind in zip(time_list,continent_color_list):

        output_filename_prefix = './tmp/'
        output_filename_extension = 'gmt'
        topology2gmt(input_rotation_filenames,
                     topology_features,
                     reconstruction_time,
                     output_filename_prefix,
                     output_filename_extension,
                     0)

        for filename in ['./tmp/subduction_boundaries_sL_%0.2fMa.gmt','./tmp/subduction_boundaries_sR_%0.2fMa.gmt']:
            features = pygplates.FeatureCollection(filename % reconstruction_time)
            new_features = []
            for feature in features:
                feature.set_valid_time(reconstruction_time,reconstruction_time)
                new_features.append(feature)
            print filename % reconstruction_time
            pygplates.FeatureCollection(new_features).write(filename % reconstruction_time)

        os.system('gmt psxy -R %s -W0.5p+cl -aZ=FROMAGE -Sf6p/1.2plt -O -K -m -Csubduction_age.cpt ./tmp/subduction_boundaries_sL_%0.2fMa.gmt -V >> %s' % (proj,reconstruction_time,outfile))
        os.system('gmt psxy -R %s -W0.5p+cl -aZ=FROMAGE -Sf6p/1.2prt -O -K -m -Csubduction_age.cpt ./tmp/subduction_boundaries_sR_%0.2fMa.gmt -V >> %s' % (proj,reconstruction_time,outfile))

    os.system('gmt psbasemap -R0/360/-90/90 %s -B45We  --MAP_TICK_LENGTH=0.1c -O -K >> %s ' % (proj,outfile))    
    os.system('gmt psclip -C -O -K >> %s ' % (outfile))



outfile='M2016_subduction_2x2.ps'


os.system('gmt pstext -R0/1/0/1 -JX1 -K -V -P <<end > %s \n 0 0  0 0 0 LM . \n end' % outfile)

#plot_M2016_map(0,100,0,6.4)
#plot_M2016_map(100,220,0,-3.2)
#plot_V2012_map(0,100,6.3,3.2)
#plot_V2012_map(100,220,0,-3.2)

plot_M2016_map(0,50,0,12.8)
plot_M2016_map(50,100,0,-3.2)
plot_M2016_map(100,150,0,-3.2)
plot_M2016_map(150,220,0,-3.2)
plot_V2012_map(0,50,6.3,9.6)
plot_V2012_map(50,100,0,-3.2)
plot_V2012_map(100,150,0,-3.2)
plot_V2012_map(150,220,0,-3.2)


os.system('gmt psscale -Csubduction_age.cpt -Dx-5.25c/-0.6+w10/0.2h+ef -Baf+l"Subduction Age (Ma)" -V -O -N --FONT_LABEL=10 --FONT_ANNOT_PRIMARY=8 --MAP_ANNOT_OFFSET_PRIMARY=0.2c --MAP_FRAME_PEN=0.9p --MAP_LABEL_OFFSET=0.1c --MAP_TICK_LENGTH=0.1c >> %s' % outfile)

#convert ps into raster, -E set the resolution
#os.system('gmt ps2raster %s -A0.2c -E300 -Tg -P' % outfile)
os.system('gmt psconvert %s -A0.2c -E300 -Tg -P -Z' % outfile)


#for filename in os.listdir('./tmp'):
#    os.remove('./tmp/%s' % filename)




[1.         0.83333333 0.66666667 0.5        0.33333333 0.16666667
 0.        ]
./tmp/subduction_boundaries_sL_50.00Ma.gmt
./tmp/subduction_boundaries_sR_50.00Ma.gmt
./tmp/subduction_boundaries_sL_40.00Ma.gmt
./tmp/subduction_boundaries_sR_40.00Ma.gmt
./tmp/subduction_boundaries_sL_30.00Ma.gmt
./tmp/subduction_boundaries_sR_30.00Ma.gmt
./tmp/subduction_boundaries_sL_20.00Ma.gmt
./tmp/subduction_boundaries_sR_20.00Ma.gmt
./tmp/subduction_boundaries_sL_10.00Ma.gmt
./tmp/subduction_boundaries_sR_10.00Ma.gmt
./tmp/subduction_boundaries_sL_0.00Ma.gmt
./tmp/subduction_boundaries_sR_0.00Ma.gmt
./tmp/subduction_boundaries_sL_100.00Ma.gmt
./tmp/subduction_boundaries_sR_100.00Ma.gmt
./tmp/subduction_boundaries_sL_90.00Ma.gmt
./tmp/subduction_boundaries_sR_90.00Ma.gmt
./tmp/subduction_boundaries_sL_80.00Ma.gmt
./tmp/subduction_boundaries_sR_80.00Ma.gmt
./tmp/subduction_boundaries_sL_70.00Ma.gmt
./tmp/subduction_boundaries_sR_70.00Ma.gmt
./tmp/subduction_boundaries_sL_60.00Ma.gmt
./tmp/subduction_

0